In [2]:
#Check if the folder exists, if not create a new path, improve code stability
import os
import numpy as np
import pandas as pd
import zipfile

from urllib.request import urlretrieve

#Go back one level from notebook to data 
output_relative_dir = '../data/'

if not os.path.exists(output_relative_dir):
    os.makedirs(output_relative_dir)
    
for target_dir in ('curated','outer'):
    if not os.path.exists(output_relative_dir + target_dir):
        os.makedirs(output_relative_dir + target_dir)

In [3]:
url = "https://www.abs.gov.au/census/find-census-data/datapacks/download/2021_GCP_SA2_for_AUS_short-header.zip"#year-month.parquet
output_dir = "../data/outer/census.zip"
# download
urlretrieve(url, output_dir) 
print("complete")

complete


In [4]:
files = zipfile.ZipFile('../data/outer/census.zip','r')

for file in files.namelist():
    files.extract(file, f"../data/outer/census")

In [5]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Tutorial 1")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)

22/09/05 13:52:06 WARN Utils: Your hostname, hexiangyideMacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.13.188.75 instead (on interface en0)
22/09/05 13:52:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/05 13:52:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
G04= spark.read.option("header",True).csv('../data/outer/census/2021 Census GCP Statistical Area 2 for AUS/2021Census_G17A_AUST_SA2.csv') #read the parquet 
G = G04.na.drop(subset=G04.columns)
print(G04.count(),G.count())

22/09/05 13:52:15 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
2472 2472


In [12]:
url = "http://github.com/matthewproctor/australianpostcodes/zipball/master"#year-month.parquet
output_dir = "../data/outer/total_pto2.zip"
# download
urlretrieve(url, output_dir) 
print("complete")


complete


22/09/05 14:12:22 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 285553 ms exceeds timeout 120000 ms
22/09/05 14:12:22 WARN SparkContext: Killing executors is not supported by current scheduler.


In [13]:
files = zipfile.ZipFile('../data/outer/total_pto2.zip','r')
for file in files.namelist():
    files.extract(file, f"../data/outer/total_pto2")

In [18]:
post_sa2= spark.read.option("header",True).csv('../data/outer/total_pto2/matthewproctor-australianpostcodes-6f8a994/australian_postcodes.csv') #read the parquet 
post_sa2

id,postcode,locality,state,long,lat,dc,type,status,sa3,sa3name,sa4,sa4name,region,Lat_precise,Long_precise,SA1_MAINCODE_2011,SA1_MAINCODE_2016,SA2_MAINCODE_2016,SA2_NAME_2016,SA3_CODE_2016,SA3_NAME_2016,SA4_CODE_2016,SA4_NAME_2016,RA_2011,RA_2016,MMM_2015,MMM_2019,ced,altitude,chargezone,phn_code,phn_name,lgaregion,electorate,electoraterating
230,0200,ANU,ACT,149.119,-35.2777,null,null,null,null,null,null,null,R1,-35.2777,149.119,80105104901,80105104901,801051049,Acton,80105,North Canberra,801,Australian Capita...,1,1,1,1,null,null,N2,null,null,null,Durack,null
21820,0200,Australian Nation...,ACT,149.1189,-35.2777,null,null,Added 19-Jan-2020,null,null,null,null,R1,-35.2776999,149.118527,80105104901,80105104901,801051049,Acton,80105,North Canberra,801,Australian Capita...,1,1,1,1,null,null,N2,null,null,null,Durack,null
232,0800,DARWIN,NT,130.83668,-12.458684,null,null,Updated 6-Feb-2020,70101,Darwin City,701,Darwin,R1,-12.3932794,130.7766611,70101100203,70101100218,701011002,Darwin City,70101,Darwin City,701,Darwin,3,3,2,2,null,null,NT1,PHN701,Northern Territory,Darwin,Solomon,Inner Metropolitan
24049,0800,DARWIN CITY,NT,130.83668,-12.458684,null,null,Updated 6-Feb-2020,70101,Darwin City,701,Darwin,R1,-12.3932794,130.7766611,70101100203,70101100218,701011002,Darwin City,70101,Darwin City,701,Darwin,3,3,2,2,null,null,NT1,PHN701,Northern Territory,Darwin,Solomon,Inner Metropolitan
233,0801,DARWIN,NT,130.83668,-12.458684,null,null,Updated 25-Mar-20...,70101,Darwin City,701,Darwin,R1,-12.4634403,130.8456418,70101100208,70101100208,701011002,Darwin City,70101,Darwin City,701,Darwin,3,3,2,2,null,null,NT1,PHN701,null,null,Lingiari,Rural
234,0804,PARAP,NT,130.873315,-12.428017,null,null,Updated 25-Mar-20...,70102,Darwin Suburbs,701,Darwin,R1,-12.4324801,130.8462536,70101100704,70101100704,701011007,Parap,70101,Darwin City,701,Darwin,3,3,2,2,null,null,NT1,PHN701,null,null,Durack,null
235,0810,ALAWA,NT,130.866242,-12.381806,null,null,Updated 6-Feb-2020,70102,Darwin Suburbs,701,Darwin,R1,-12.38,130.873,70102101001,70102101001,701021010,Alawa,70102,Darwin Suburbs,701,Darwin,3,3,2,2,null,9.22142887115479,NT1,PHN701,Northern Territory,Darwin,Solomon,Inner Metropolitan
236,0810,BRINKIN,NT,130.866242,-12.381806,null,null,Updated 6-Feb-2020,70102,Darwin Suburbs,701,Darwin,R1,-12.37,130.867,70102101301,70102101301,701021013,Brinkin - Nakara,70102,Darwin Suburbs,701,Darwin,3,3,2,2,null,9.22142887115479,NT1,PHN701,Northern Territory,Darwin,Solomon,Inner Metropolitan
237,0810,CASUARINA,NT,130.866242,-12.381806,null,null,Updated 6-Feb-2020,70102,Darwin Suburbs,701,Darwin,R1,-12.373333,130.881667,70102101307,70102101307,701021013,Brinkin - Nakara,70102,Darwin Suburbs,701,Darwin,3,3,2,2,null,9.22142887115479,NT1,PHN701,Northern Territory,Darwin,Solomon,Inner Metropolitan
19972,0810,COCONUT GROVE,NT,130.8502369,-12.39614166,Northern Territor...,Delivery Area,Updated 6-Feb-2020,70102,Darwin Suburbs,701,Darwin,R1,-12.3989,130.852,70102101604,70102101604,701021016,Coconut Grove,70102,Darwin Suburbs,701,Darwin,3,3,2,2,null,9.22142887115479,NT1,PHN701,Northern Territory,Darwin,Solomon,Inner Metropolitan


In [ ]:
#K = post_sa2.na.drop(subset=post_sa2.columns)
print(post_sa2.count(),K.count())